In [1]:
import spotlight
annotations = spotlight.annotate('https://api.dbpedia-spotlight.org/en/annotate',
    'Berlin is the capital and largest city of Germany',confidence=0.4,support = 20)
annotations

[{'URI': 'http://dbpedia.org/resource/Berlin',
  'support': 87107,
  'types': 'Wikidata:Q515,Wikidata:Q486972,Schema:Place,Schema:City,DBpedia:Settlement,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:City',
  'surfaceForm': 'Berlin',
  'offset': 0,
  'similarityScore': 0.9983656061097194,
  'percentageOfSecondRank': 0.0008228013211064525},
 {'URI': 'http://dbpedia.org/resource/City',
  'support': 22610,
  'types': '',
  'surfaceForm': 'city',
  'offset': 34,
  'similarityScore': 0.8720405806057635,
  'percentageOfSecondRank': 0.10547609042001196},
 {'URI': 'http://dbpedia.org/resource/Germany',
  'support': 222752,
  'types': 'Wikidata:Q6256,Schema:Place,Schema:Country,DBpedia:PopulatedPlace,DBpedia:Place,DBpedia:Location,DBpedia:Country',
  'surfaceForm': 'Germany',
  'offset': 42,
  'similarityScore': 0.9959557085639706,
  'percentageOfSecondRank': 0.0024005027968948783}]

In [2]:
annotations[0]["URI"]

'http://dbpedia.org/resource/Berlin'

In [3]:
def getURIs(json):
    URIs = []
    for obj in json:
        URIs.append(obj["URI"])

    return URIs
URIs = getURIs(annotations)

In [4]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
endpoint = "https://dbpedia.org/sparql"
def sparql_to_dataframe(endpoint, query):
    """
    Convert SPARQL results into a Pandas DataFrame.
    Credit: https://lawlesst.github.io/notebook/sparql-dataframe.html
    """
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()

    cols = results['head']['vars']
    out = []
    for row in results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)


In [5]:
def test(URI):
    return '''
select * where {
    <%s> rdfs:comment ?y.
    filter(lang(?y)="en")
 }'''%(URI)

In [6]:
def retrieveTest(endpoint,uri) :
    query = test(uri)
    result = sparql_to_dataframe(endpoint, query)
    return result

In [7]:
retrieveTest(endpoint,URIs[0])

,y
0,"Berlin (/bɜːrˈlɪn/ bur-LIN, German: [bɛʁˈliːn]..."


In [8]:
df_train = pd.read_csv('data_train.csv')
df_train.head()

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0


In [9]:
df_test = pd.read_csv('data_test.csv')
df_test.head()

,text,label
0,Analyst call of the day for @CNBCPro subscribe...,0
1,"Loop upgrades CSX to buy, says it's a good pla...",0
2,BofA believes we're already in a recession — a...,0
3,JPMorgan sees these derivative plays as best w...,0
4,Morgan Stanley's Huberty sees Apple earnings m...,0


In [10]:
df_test = df_test.iloc[:,0]
df_test.head()

0    Analyst call of the day for @CNBCPro subscribe...
1    Loop upgrades CSX to buy, says it's a good pla...
2    BofA believes we're already in a recession — a...
3    JPMorgan sees these derivative plays as best w...
4    Morgan Stanley's Huberty sees Apple earnings m...
Name: text, dtype: object

In [12]:
for index, row in df_test.iterrows():
    text_to_annotate = row['text']
    annotations = spotlight.annotate(text_to_annotate, confidence=0.4, support=20)
    print(annotations)


AttributeError: 'Series' object has no attribute 'iterrows'